In [26]:
!pip install torch torchvision pytorch_lightning

Качаем данные

In [28]:
import torch
import pytorch_lightning as pl
from torch import nn
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn import datasets

Преобработка данных

In [29]:
iris = datasets.load_iris()
x,y = iris.data, iris.target
scaler = StandardScaler()
x = scaler.fit_transform(x)
le = LabelEncoder()
y = le.fit_transform(y)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(x_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))

In [30]:
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = LabelEncoder().fit_transform(y)

In [31]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(x_val, dtype=torch.float32),
                            torch.tensor(y_val, dtype=torch.long))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

Начинаем писать класс перцептрона на PL

In [32]:
class Perceptron(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_neurons=10, learning_rate=0.01):
        super().__init__()
        self.hidden_neurons = hidden_neurons
        self.learning_rate = learning_rate

        self.weights = torch.nn.Parameter(torch.randn(input_dim, hidden_neurons))#Входные
        self.bias = torch.nn.Parameter(torch.zeros(hidden_neurons))

        self.output_weights = torch.nn.Parameter(torch.randn(hidden_neurons, output_dim))#Выходыне
        self.output_bias = torch.nn.Parameter(torch.zeros(output_dim))

    def forward(self, x):
        z = torch.matmul(x, self.weights) + self.bias#Что-то типа ax+b, т.е. умножаем входные данные на веса
        hidden_output = torch.sigmoid(z)#Сглаживаем от 0 жо 1
        y_pred = torch.matmul(hidden_output, self.output_weights) + self.output_bias#Все то же самое, только уже на скрытый выход
        return torch.softmax(y_pred, dim=1)#Возвращаем вероятности, нормализуя вдоль 2 оси

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)#Прогоняем шаг от одного батча
        loss = self.cross_entropy_loss(y_pred, y)#Считаем потери
        accuracy = self.compute_accuracy(y_pred, y)#Считаем точность

        self.log('train_loss', loss, prog_bar=True)
        self.log('train_accuracy', accuracy, prog_bar=True)#Просто логируем в консоль их
        self.manual_backprop(x, y, y_pred)#Вызов функции расчета градиента вручную
        return loss

    def configure_optimizers(self):
        return None  #Не используется

    def cross_entropy_loss(self, y_pred, y):
        y_one_hot = torch.nn.functional.one_hot(y, num_classes=y_pred.size(1)).float()#Просто переделываем в формат [0,0,1], где единица - принадлежность классу
                                                                                #количество классов равно количеству колонок в y_pred
        return -(y_one_hot * torch.log(y_pred + 1e-9)).sum(dim=1).mean()#Чтобы логарифм не ушел от 0 - добавляем 1e-9. Затем поэлементно умножаем уже преобразованный в onehot с логарифмами
                                                                        #предсказаний, суммируем по строчкам,выдем среднее

    def compute_accuracy(self, y_pred, y):
        predicted_classes = torch.argmax(y_pred, dim=1)
        accuracy = (predicted_classes == y).float().mean()#Просто считаем точность
        return accuracy

    def manual_backprop(self, x, y, y_pred):
        y_one_hot = torch.nn.functional.one_hot(y, num_classes=y_pred.size(1)).float()#Преобразоывыве в onehot настоящие лэйблы

        #Выходые веса
        output_error = y_pred - y_one_hot #Считаем разброс
        hidden_output = torch.sigmoid(torch.matmul(x, self.weights) + self.bias)#Так же преобразовываем после перемножения входа на весы + смещение
        grad_output_weights = torch.matmul(hidden_output.T, output_error) / x.size(0)#перемножаем ранее высчитанный разброс на транспонированную матрицу скрытого выхода + для нормализации делим на размерность батча
        grad_output_bias = output_error.mean(dim=0) #Считаем среднее для градиента свободных членов

        #Скрытые веса
        def_sigm = hidden_output * (1 - hidden_output) # производная функции активации
        hidden_error = torch.matmul(output_error, self.output_weights.T) * def_sigm#Умножаем ошибку на выходе на траспонировнные веса выхода, а затем поэлементно на производную
        grad_weights = torch.matmul(x.T, hidden_error) / x.size(0)#Градиент свободных членов для скрытых путем матричного перемножения ошибки на на скрытом слое на транспонировнный вход
        grad_bias = hidden_error.mean(dim=0)#Градиент для свободных скрытых

        #Обновляем веса и смещения путем вычитания из них нвого градиента, умноженного на скорость обучения
        self.output_weights.data -= self.learning_rate * grad_output_weights
        self.output_bias.data -= self.learning_rate * grad_output_bias
        self.weights.data -= self.learning_rate * grad_weights
        self.bias.data -= self.learning_rate * grad_bias

Обучение

In [33]:
nums_of_neurons = 50
lr = 0.01
num_epoch = 120
log_steps = 1

In [34]:
model = Perceptron(input_dim=x.shape[1],
                   output_dim=len(set(y)),
                   hidden_neurons=nums_of_neurons,
                   learning_rate=lr)
trainer = pl.Trainer(max_epochs=num_epoch,
                     enable_checkpointing=False,
                     log_every_n_steps=log_steps)
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 403    | n/a 
---------------------------------------------
403       Trainable params
0         Non-trainable params
403       Total params
0.002     Total estimated model params size (MB)
0         Modules in train mode
0         Modules in eval

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=120` reached.
